In [ ]:
# DNN with 1025 input and 2050 output

In [1]:
#import libraries.
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
# import sounddevice as sd
import tensorflow as tf
# from separation import bss_eval_sources 
from pystoi.stoi import stoi 

Using TensorFlow backend.


In [2]:
#import libraries.
import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K

In [3]:
#read necessary files, contarins mixed log power features.
temp = open( "/home/majid/Codes/Thesis/TIMIT/Organized/concatenated/Mixed0.p", "rb")
ftr = pickle.load( temp  ) .T
temp.close()
#contarins target(clean) log power features.
temp = open( "/home/majid/Codes/Thesis/TIMIT/Organized/concatenated/clean0.p", "rb")
target = pickle.load( temp ).T
temp.close()

In [44]:
ftr.shape

(116351, 1025)

In [4]:
X = ftr[0:100000]
y = target[0:100000]
X_test = ftr[100000:]
Y_test = target[100000:]


In [5]:
h = [1025,1025,1025]

In [6]:
ftr = []
target = []

In [7]:
def DNN_A():
        # create model
        model = Sequential()
        model.add(Dense(h[0], input_dim = X.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(h[2], kernel_initializer='normal', activation='relu'))
        model.add(Dense(y.shape[1], kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model

In [8]:
seed = 1
np.random.seed(seed)
estimator_A = KerasRegressor(build_fn=DNN_A, epochs=10, batch_size=150, verbose=1)
kfold = KFold(n_splits=2, random_state=seed)
results = cross_val_score(estimator_A, X, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
estimator_A.fit(X, y)
prediction = estimator_A.predict(X_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
50000/50000 [==============================] - 27s 533us/step - loss: 0.1703
Epoch 2/10
50000/50000 [==============================] - 25s 505us/step - loss: 0.1228
Epoch 3/10
50000/50000 [==============================] - 25s 508us/step - loss: 0.1131
Epoch 4/10
50000/50000 [==============================] - 26s 515us/step - loss: 0.1057
Epoch 5/10
50000/50000 [==============================] - 26s 519us/step - loss: 0.0987
Epoch 6/10
50000/50000 [==============================] - 26s 528us/step - loss: 0.0932
Epoch 7/10
50000/50000 [==============================] - 26s 529us/step - loss: 0.0875
Epoch 8/10
50000/50000 [==============================] - 26s 523us/step - loss: 0.0819
Epoch 9/10
50000/50000 [==============================] - 26s 511us/step - loss: 0.0773
Epoch 10/10
50000/50000 [==============================] - 5s 101us/step
Epoch 1/10
5000

In [11]:
# Save a model you have trained
estimator_A.model.save('Models/Two_stage/trained_model.h5')

In [16]:
print(a)

<_io.BufferedReader name='Models/Two_stage/trained_model.h5'>


In [17]:
#read necessary files, contarins mixed log power features.
temp = open( "/home/majid/Codes/Thesis/TIMIT/Organized/concatenated/Mixed1.p", "rb")
ftr = pickle.load( temp  ) .T
temp.close()
#contarins target(clean) log power features.
temp = open( "/home/majid/Codes/Thesis/TIMIT/Organized/concatenated/clean1.p", "rb")
target = pickle.load( temp ).T
temp.close()

In [18]:
X = ftr[0:10000]
y = target[0:10000]
X_test = ftr[100000:]
Y_test = target[100000:]

In [19]:
ftr = []
target = []

In [21]:
# Load the model
from keras.models import load_model
estimator_A = load_model('Models/Two_stage/trained_model.h5')

In [22]:
# fit the model
estimator_A.fit(X, y, epochs=10, batch_size=150)

Epoch 1/10
10000/10000 [==============================] - 6s 601us/step - loss: 0.1140
Epoch 2/10
10000/10000 [==============================] - 6s 627us/step - loss: 0.0911
Epoch 3/10
10000/10000 [==============================] - 6s 597us/step - loss: 0.0804
Epoch 4/10
10000/10000 [==============================] - 6s 593us/step - loss: 0.0725
Epoch 5/10
10000/10000 [==============================] - 6s 589us/step - loss: 0.0668
Epoch 6/10
10000/10000 [==============================] - 6s 570us/step - loss: 0.0620
Epoch 7/10
10000/10000 [==============================] - 6s 571us/step - loss: 0.0586
Epoch 8/10
10000/10000 [==============================] - 6s 572us/step - loss: 0.0549
Epoch 9/10
10000/10000 [==============================] - 6s 569us/step - loss: 0.0524
Epoch 10/10
10000/10000 [==============================] - 6s 571us/step - loss: 0.0503


In [22]:
# from keras.callbacks import ModelCheckpoint
# # define the checkpoint
# filepath = "model.h5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]

In [23]:
# # fit the model
# estimator_A.fit(X[300:600], y[300:600], epochs=10, batch_size=150, callbacks=callbacks_list)

Epoch 1/10
300/300 [==============================] - 1s 3ms/step - loss: 1.6880

Epoch 00001: loss improved from inf to 1.68803, saving model to model.h5
Epoch 2/10
300/300 [==============================] - 0s 555us/step - loss: 0.2899

Epoch 00002: loss improved from 1.68803 to 0.28990, saving model to model.h5
Epoch 3/10
300/300 [==============================] - 0s 540us/step - loss: 0.2072

Epoch 00003: loss improved from 0.28990 to 0.20718, saving model to model.h5
Epoch 4/10
300/300 [==============================] - 0s 625us/step - loss: 0.1929

Epoch 00004: loss improved from 0.20718 to 0.19287, saving model to model.h5
Epoch 5/10
300/300 [==============================] - 0s 514us/step - loss: 0.1905

Epoch 00005: loss improved from 0.19287 to 0.19050, saving model to model.h5
Epoch 6/10
300/300 [==============================] - 0s 508us/step - loss: 0.1889

Epoch 00006: loss improved from 0.19050 to 0.18888, saving model to model.h5
Epoch 7/10
300/300 [====================

In [25]:
# fit the model
# estimator_A.fit(X[300:600], y[300:600])

Epoch 1/10
300/300 [==============================] - 1s 3ms/step - loss: 1.6880
Epoch 2/10
300/300 [==============================] - 0s 542us/step - loss: 0.2993
Epoch 3/10
300/300 [==============================] - 0s 567us/step - loss: 0.2086
Epoch 4/10
300/300 [==============================] - 0s 570us/step - loss: 0.1932
Epoch 5/10
300/300 [==============================] - 0s 592us/step - loss: 0.1905
Epoch 6/10
300/300 [==============================] - 0s 577us/step - loss: 0.1889
Epoch 7/10
300/300 [==============================] - 0s 578us/step - loss: 0.1864
Epoch 8/10
300/300 [==============================] - 0s 586us/step - loss: 0.1829
Epoch 9/10
300/300 [==============================] - 0s 576us/step - loss: 0.1787
Epoch 10/10
300/300 [==============================] - 0s 639us/step - loss: 0.1741
